## 데이터 불러오기

In [2]:
pip install nltk

In [41]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4")

FHP_article = pd.read_json('healthcare_project_FHP_article.json')
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,title,date,abstract
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...
...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an..."


## 데이터 전처리

### 결측치 처리
- abstract가 없는 값은 제목으로 대체

In [9]:
FHP_article.query("abstract==''").count()

title       9
date        9
abstract    9
dtype: int64

In [10]:
FHP_article.loc[FHP_article.query("abstract==''").index,'abstract'] = FHP_article.loc[FHP_article.query("abstract==''").index,'title']

In [11]:
FHP_article.query("abstract==''").sum()

title       0
date        0
abstract    0
dtype: object

### 발행일 변환

In [12]:
def date_year(raw):
    date_format = raw.split(" ")[0]
    return date_format
def date_month(raw):
    if len(raw.split(" ")) >= 2:
        date_format = raw.split(" ")[1]
    else:
        date_format = None
    return date_format
def date_date(raw):
    if len(raw.split(" ")) >= 3:
        date_format = raw.split(" ")[2]
    else:
        date_format = None
    return date_format
def word_split(raw,word_list):
    word_list.append(raw.split())
    return word_list

In [13]:
FHP_article.loc[:,'date_year']=FHP_article['date'].apply(date_year)
FHP_article.loc[:,'date_month']=FHP_article['date'].apply(date_month)
FHP_article.loc[:,'date_date']=FHP_article['date'].apply(date_date)

### 정규화 : 대소문자 통합

In [14]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)

In [15]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract'].apply(word_lower)

In [16]:
FHP_article.loc[2,'abstract_clean']

'background: forward head posture and rounded shoulder posture are common postural variants found in upper crossed syndrome, which can lead to limited neck mobility, respiratory problems, and other issues. the purpose of this study was to investigate the effects of telerehabilitation, combining diaphragmatic breathing re-education and shoulder stabilization exercises, on young men with upper crossed syndrome during the covid-19 pandemic over 4 weeks. methods: the study included 37 participants aged in their 20s and 30s who were randomly divided into two groups. the experimental group received diaphragmatic breathing re-education and shoulder stabilization exercises, while the control group only underwent shoulder stabilization exercises. both groups were trained three times a week for four weeks using telerehabilitation. the comparison of within-group pre-post differences in the experimental and control groups was conducted using a paired t-test, while the effects of treatment were ass

### 특수문자, 숫자, '/n', '소제목' 제거

In [17]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    raw = raw.replace("background:"," ")
    raw = raw.replace("purpose:"," ")
    raw = raw.replace("objectives:"," ")
    raw = raw.replace("objective:"," ")
    raw = raw.replace("design:"," ")
    raw = raw.replace("methods:"," ")
    raw = raw.replace("method:"," ")
    raw = raw.replace("results:"," ")
    raw = raw.replace("result:"," ")
    raw = raw.replace("discussion:"," ")
    raw = raw.replace("conclusions:"," ")
    raw = raw.replace("interpretation:"," ")
    raw = raw.replace("conclusion:"," ")
    raw = raw.replace("significance:"," ")
    raw = raw.replace("keywords:"," ")
    raw = raw.replace("fhp"," ")
    raw = raw.replace("forward head posture"," ")
    raw = raw.replace("head forward posture"," ")
    patterns = '([^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex

In [18]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_cleaning)

In [19]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...,2024,Apr,1,chronic neck pain cnp is a common public hea...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...,2024,Apr,1,the purpose of this study was to clarify the...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...,2024,Mar,11,and rounded shoulder posture are common po...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...,2024,Mar,4,the purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...,2024,Mar,26,this study aimed to compare sensory organiza...
...,...,...,...,...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...,1975,Feb,None,the hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,Effect of forward head inclination on visual o...,1972,Nov,None,effect of forward head inclination on visual o...
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,[The mechanism of elongation of the eye-axis. ...,1972,Aug,None,the mechanism of elongation of the eyeaxis i t...
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an...",1966,Mar,5,the clinical picture radiological findings and...


### 단어 토큰화

In [20]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True)
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    nouns_list = [t for t in tags if t[1] in  ['JJ','JJR','NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    return nouns_list


In [21]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

In [22]:
FHP_article['abstract_token']

0       [(chronic, JJ), (neck, NN), (pain, NN), (cnp, ...
1       [(purpose, NN), (study, NN), (was, VBD), (clar...
2       [(rounded, VBD), (shoulder, NN), (posture, NN)...
3       [(purpose, NN), (randomized, VBN), (controlled...
4       [(study, NN), (aimed, VBD), (compare, VB), (se...
                              ...                        
1246    [(hypothesis, NN), (is, VBZ), (put, VBN), (dis...
1247    [(effect, NN), (forward, JJ), (head, NN), (inc...
1248    [(mechanism, NN), (elongation, NN), (eyeaxis, ...
1249    [(clinical, JJ), (picture, NN), (radiological,...
1250    [(oscillation, NN), (eyes, NNS), (patient, NN)...
Name: abstract_token, Length: 1251, dtype: object

### 표제어 추출

In [23]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        if word[1][0].lower() in ['v','n']:
            lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
            if lemmatized_word != 'be':
                lemmatized_list.append(lemmatized_word)
        else:
            lemmatized_list.append(word[0])
    return lemmatized_list

In [24]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [25]:
FHP_article.loc[:,'abstract_token']

0       [chronic, neck, pain, cnp, common, public, hea...
1       [purpose, study, clarify, relationship, smartp...
2       [round, shoulder, posture, common, postural, v...
3       [purpose, randomize, control, trial, evaluate,...
4       [study, aim, compare, sensory, organization, t...
                              ...                        
1246    [hypothesis, put, disturbance, gait, see, tumo...
1247    [effect, forward, head, inclination, visual, o...
1248    [mechanism, elongation, eyeaxis, i, intraocula...
1249    [clinical, picture, radiological, finding, tre...
1250    [oscillation, eye, patient, head, place, certa...
Name: abstract_token, Length: 1251, dtype: object

In [26]:
def join_words(raw):
    sentence = ' '.join(raw)
    return sentence

In [27]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)

In [28]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...,2024,Apr,1,chronic neck pain cnp is a common public hea...,"[chronic, neck, pain, cnp, common, public, hea...",chronic neck pain cnp common public health pro...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...,2024,Apr,1,the purpose of this study was to clarify the...,"[purpose, study, clarify, relationship, smartp...",purpose study clarify relationship smartphone ...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...,2024,Mar,11,and rounded shoulder posture are common po...,"[round, shoulder, posture, common, postural, v...",round shoulder posture common postural variant...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...,2024,Mar,4,the purpose of this randomized controlled tria...,"[purpose, randomize, control, trial, evaluate,...",purpose randomize control trial evaluate effec...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...,2024,Mar,26,this study aimed to compare sensory organiza...,"[study, aim, compare, sensory, organization, t...",study aim compare sensory organization test po...
...,...,...,...,...,...,...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...,1975,Feb,None,the hypothesis is put forward that the disturb...,"[hypothesis, put, disturbance, gait, see, tumo...",hypothesis put disturbance gait see tumour pos...
1247,Effect of forward head inclination on visual o...,1972 Nov,Effect of forward head inclination on visual o...,1972,Nov,None,effect of forward head inclination on visual o...,"[effect, forward, head, inclination, visual, o...",effect forward head inclination visual orienta...
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,[The mechanism of elongation of the eye-axis. ...,1972,Aug,None,the mechanism of elongation of the eyeaxis i t...,"[mechanism, elongation, eyeaxis, i, intraocula...",mechanism elongation eyeaxis i intraocular pre...
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an...",1966,Mar,5,the clinical picture radiological findings and...,"[clinical, picture, radiological, finding, tre...",clinical picture radiological finding treatmen...


### 유사한 단어 묶기

In [29]:
def word_cleaning(raw):
    raw = raw.replace("chronic neck pain","cnp")
    return raw

## 키워드 분류하기

### TfidfVectorizer 최적화

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(max_df=0.60,min_df=0.2,ngram_range=(1,2), stop_words = ['use','have','show'])
features = tfidfVectorizer.fit_transform(FHP_article['abstract_join'])
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['age', 'aim', 'analysis', 'angle', 'assess', 'body', 'cervical',
       'change', 'compare', 'control', 'determine', 'difference',
       'different', 'effect', 'evaluate', 'find', 'forward', 'group',
       'include', 'increase', 'investigate', 'measure', 'movement',
       'muscle', 'neck', 'pain', 'patient', 'perform', 'position',
       'postural', 'posture', 'result', 'significant', 'subject',
       'suggest', 'test', 'upper', 'year'], dtype=object)

### 토픽모델링

In [39]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3)
lda.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
topics_list = list()
for topic in lda.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,position body movement subject change forward posture muscle postural result
1,Topic2,neck cervical pain posture muscle patient angle upper include increase
2,Topic3,group control difference angle patient significant test age compare postural


In [43]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(lda,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
topics_output = lda.transform(features)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dominant_topic_number
0    496
2    392
1    363
Name: count, dtype: int64

## 토픽 분석하기

### topic별로 분류하기

In [46]:
FHP_article['article_topic'] = df_topics_score['dominant_topic_number']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
FHP_article

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
0,Effects of scapular treatment on chronic neck pain: a systematic review and meta-analysis of randomized controlled trials,2024 Apr 1,"Background: Chronic neck pain (CNP) is a common public health problem that affects daily living activities and quality of life. There is biomechanical interdependence between the neck and scapula. Studies have shown that shoulder blade function might be related to chronic neck pain. We therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with CNP.\nMethods: Databases, including MEDLINE (via PubMed), EMBASE (via Ovid), Ovid, Web of Science, and Scopus, were systematically searched for randomized controlled trials published in English investigating treatment of the scapula for CNP before July 16, 2023.\nResults: A total of 313 participants were included from 8 RCTs. Compared with those in the control group, the intervention in the scapular treatment group exhibited greater improvement in pain intensity (standardized mean difference (SMD) = 2.55; 95% CI = 0.97 to 4.13; P = 0.002), with moderate evidence. Subgroup analysis for pain intensity revealed a significant difference between the sexes, with only the female population (SMD = 6.23, 95% CI = 4.80 to 7.65) showing better outcomes than those with both sexes (SMD = 1.07, 95% CI = 0.57 to 1.56) (p < 0.00001). However, moderate evidence demonstrated no improvement in neck disability after scapular treatment (SMD of 0.24[-0.14, 0.62] of Neck Disability Index or Northwick Park Neck Pain Questionnaire). No effect of scapular treatment was shown on the pressure pain threshold (PPT). The cervical range of motion (CROM) and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles, and further study was needed. However, the patient's head forward posture appeared to be corrected after scapular treatment.\nConclusion: Scapular therapy was beneficial for relieving pain intensity in patients with CNP, especially in women. Head forward posture might also be corrected with scapular therapy. However, scapular therapy may have no effect on the PPT or neck disability. However, whether scapular therapy could improve CROM and cervical muscle activation in patients with CNPs had not been determined and needed further study.",2024,Apr,1,chronic neck pain cnp is a common public health problem that affects daily living activities and quality of life there is biomechanical interdependence between the neck and scapula studies have shown that shoulder blade function might be related to chronic neck pain we therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with cnp databases including medline via pubmed embase via ovid ovid web of science and scopus were systematically searched for randomized controlled trials published in english investigating treatment of the scapula for cnp before july a total of participants were included from rcts compared with those in the control group the intervention in the scapular treatment group exhibited greater improvement in pain intensity standardized mean difference smd ci to p with moderate evidence subgroup analysis for pain intensity revealed a significant difference between the sexes with only the female population smd ci to showing better outcomes than those with both sexes smd ci to p however moderate evidence demonstrated no improvement in neck disability after scapular treatment smd of of neck disability index or northwick park neck pain questionnaire no effect of scapular treatment was shown on the pressure pain threshold ppt the cervical range of motion crom and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles and further study was needed however the patient's appeared to be corrected after scapular 

### topic 별 많이 등장하는 단어 추출

In [48]:
FHP_article.query("article_topic == 0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
7,Influence of oral motor tasks on postural muscle activity during dynamic reactive balance,2024 Mar 15.,"Background: Jaw clenching improves dynamic reactive balance on an oscillating platform during forward acceleration and is associated with decreased mean sway speed of different body regions.\nObjective: It is suggested that jaw clenching as a concurrent muscle activity facilitates human motor excitability, increasing the neural drive to distal muscles. The underlying mechanism behind this phenomenon was studied based on leg and trunk muscle activity (iEMG) and co-contraction ratio (CCR).\nMethods: Forty-eight physically active and healthy adults were assigned to three groups, performing three oral motor tasks (jaw clenching, tongue pressing against the palate or habitual lower jaw position) during a dynamic one-legged stance reactive balance task on an oscillating platform. The iEMG and CCR of posture-relevant muscles and muscle pairs were analysed during platform forward acceleration.\nResults: Tongue pressing caused an adjustment of co-contraction patterns of distal muscle groups based on changes in biomechanical coupling between the head and trunk during static balancing at the beginning of the experiment. Neither iEMG nor CCR measurement helped detect a general neuromuscular effect of jaw clenching on the dynamic reactive balance.\nConclusion: The findings might indicate the existence of robust fixed patterns of rapid postural responses during the important initial phases of balance recovery.",2024,Mar,15.,jaw clenching improves dynamic reactive balance on an oscillating platform during forward acceleration and is associated with decreased mean sway speed of different body regions it is suggested that jaw clenching as a concurrent muscle activity facilitates human motor excitability increasing the neural drive to distal muscles the underlying mechanism behind this phenomenon was studied based on leg and trunk muscle activity iemg and cocontraction ratio ccr fortyeight physically active and healthy adults were assigned to three groups performing three oral motor tasks jaw clenching tongue pressing against the palate or habitual lower jaw position during a dynamic onelegged stance reactive balance task on an oscillating platform the iemg and ccr of posturerelevant muscles and muscle pairs were analysed during platform forward acceleration tongue pressing caused an adjustment of cocontraction patterns of distal muscle groups based on changes in biomechanical coupling between the head and trunk during static balancing at the beginning of the experiment neither iemg nor ccr measurement helped detect a general neuromuscular effect of jaw clenching on the dynamic reactive balance the findings might indicate the existence of robust fixed patterns of rapid postural responses during the important initial phases of balance recovery,"[jaw, clench, improves, dynamic, reactive, balance, oscillating, platform, forward, acceleration, associate, decreased, mean, sway, speed, different, body, region, suggest, jaw, clenching, concurrent, muscle, activity, facilitate, human, motor, excitability, increase, neural, drive, distal, muscle, underlie, mechanism, phenomenon, study, base, leg, trunk, muscle, activity, iemg, cocontraction, ratio, ccr, fortyeight, active, healthy, adult, assign, group, perform, oral, motor, task, jaw, clench, tongue, press, palate, habitual, lower, jaw, position, dynamic, onelegged, stance, reactive, balance, task, oscillate, platform, iemg, ccr, posturerelevant, muscle, muscle, pair, analyse, platform, acceleration, tongue, press, cause, adjustment, cocontraction, pattern, distal, muscle, group, base, change, biomechanical, coupling, head, trunk, static, balancing, beginning, ...]",jaw clench improves dynamic reactive balance oscillating platform forward acceleration associate decreased mean sway speed dif

In [49]:
FHP_article.query("article_topic == 1")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
0,Effects of scapular treatment on chronic neck pain: a systematic review and meta-analysis of randomized controlled trials,2024 Apr 1,"Background: Chronic neck pain (CNP) is a common public health problem that affects daily living activities and quality of life. There is biomechanical interdependence between the neck and scapula. Studies have shown that shoulder blade function might be related to chronic neck pain. We therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with CNP.\nMethods: Databases, including MEDLINE (via PubMed), EMBASE (via Ovid), Ovid, Web of Science, and Scopus, were systematically searched for randomized controlled trials published in English investigating treatment of the scapula for CNP before July 16, 2023.\nResults: A total of 313 participants were included from 8 RCTs. Compared with those in the control group, the intervention in the scapular treatment group exhibited greater improvement in pain intensity (standardized mean difference (SMD) = 2.55; 95% CI = 0.97 to 4.13; P = 0.002), with moderate evidence. Subgroup analysis for pain intensity revealed a significant difference between the sexes, with only the female population (SMD = 6.23, 95% CI = 4.80 to 7.65) showing better outcomes than those with both sexes (SMD = 1.07, 95% CI = 0.57 to 1.56) (p < 0.00001). However, moderate evidence demonstrated no improvement in neck disability after scapular treatment (SMD of 0.24[-0.14, 0.62] of Neck Disability Index or Northwick Park Neck Pain Questionnaire). No effect of scapular treatment was shown on the pressure pain threshold (PPT). The cervical range of motion (CROM) and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles, and further study was needed. However, the patient's head forward posture appeared to be corrected after scapular treatment.\nConclusion: Scapular therapy was beneficial for relieving pain intensity in patients with CNP, especially in women. Head forward posture might also be corrected with scapular therapy. However, scapular therapy may have no effect on the PPT or neck disability. However, whether scapular therapy could improve CROM and cervical muscle activation in patients with CNPs had not been determined and needed further study.",2024,Apr,1,chronic neck pain cnp is a common public health problem that affects daily living activities and quality of life there is biomechanical interdependence between the neck and scapula studies have shown that shoulder blade function might be related to chronic neck pain we therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with cnp databases including medline via pubmed embase via ovid ovid web of science and scopus were systematically searched for randomized controlled trials published in english investigating treatment of the scapula for cnp before july a total of participants were included from rcts compared with those in the control group the intervention in the scapular treatment group exhibited greater improvement in pain intensity standardized mean difference smd ci to p with moderate evidence subgroup analysis for pain intensity revealed a significant difference between the sexes with only the female population smd ci to showing better outcomes than those with both sexes smd ci to p however moderate evidence demonstrated no improvement in neck disability after scapular treatment smd of of neck disability index or northwick park neck pain questionnaire no effect of scapular treatment was shown on the pressure pain threshold ppt the cervical range of motion crom and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles and further study was needed however the patient's appeared to be corrected after scapular 

In [50]:
FHP_article.query("article_topic == 2")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
2,"Effects of Telerehabilitation Combining Diaphragmatic Breathing Re-Education and Shoulder Stabilization Exercises on Neck Pain, Posture, and Function in Young Adult Men with Upper Crossed Syndrome: A Randomized Controlled Trial",2024 Mar 11,"Background: Forward head posture and rounded shoulder posture are common postural variants found in upper crossed syndrome, which can lead to limited neck mobility, respiratory problems, and other issues. The purpose of this study was to investigate the effects of telerehabilitation, combining diaphragmatic breathing re-education and shoulder stabilization exercises, on young men with upper crossed syndrome during the COVID-19 pandemic over 4 weeks. Methods: The study included 37 participants aged in their 20s and 30s who were randomly divided into two groups. The experimental group received diaphragmatic breathing re-education and shoulder stabilization exercises, while the control group only underwent shoulder stabilization exercises. Both groups were trained three times a week for four weeks using telerehabilitation. The comparison of within-group pre-post differences in the experimental and control groups was conducted using a paired t-test, while the effects of treatment were assessed using repeated-measures analysis of variance. Results: After 4 weeks, both groups showed significant improvements in the pain pressure threshold of the upper trapezius, craniovertebral angle, round shoulder posture, shoulder tilt degree, neck disability index, and closed kinetic chain upper extremity stability test (all p < 0.05). The results showed a significant difference between the Time effect (p adj < 0.05/4) for both sides of PPT, CVA, and STD and both sides of RSP, NDI, and CKCUEST, and an interaction between the Time × Group effects (p adj < 0.05/4) for the Rt. PPT, CVA, and STD. Conclusions: These findings suggest that the telerehabilitation training group, which included diaphragmatic breathing re-education and shoulder stabilization exercises, was more effective in improving Rt. PPT, CVA, and STD in males with UCS.",2024,Mar,11,and rounded shoulder posture are common postural variants found in upper crossed syndrome which can lead to limited neck mobility respiratory problems and other issues the purpose of this study was to investigate the effects of telerehabilitation combining diaphragmatic breathing reeducation and shoulder stabilization exercises on young men with upper crossed syndrome during the covid pandemic over weeks the study included participants aged in their s and s who were randomly divided into two groups the experimental group received diaphragmatic breathing reeducation and shoulder stabilization exercises while the control group only underwent shoulder stabilization exercises both groups were trained three times a week for four weeks using telerehabilitation the comparison of withingroup prepost differences in the experimental and control groups was conducted using a paired ttest while the effects of treatment were assessed using repeatedmeasures analysis of variance after weeks both groups showed significant improvements in the pain pressure threshold of the upper trapezius craniovertebral angle round shoulder posture shoulder tilt degree neck disability index and closed kinetic chain upper extremity stability test all p the results showed a significant difference between the time effect p adj for both sides of ppt cva and std and both sides of rsp ndi and ckcuest and an interaction between the time group effects p adj for the rt ppt cva and std these findings suggest that the telerehabilitation training group which included diaphragmatic breathing reeducation and shoulder stabilization exercises was more effective in improving rt ppt cva and std in males with ucs,"[round, shoulder, posture, common, postural, variant, find, upper, cross, syndrome, lead, limited,

In [51]:
FHP_article.to_csv('FHP_article.csv',index=False,encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 결론
- 토픽 별로 논문 초록을 확인해 본 결과, 거북목과 관련되지 않은 논문도 많이 존재함
- 논문을 검색할 때 거북목이 무조건 들어가 있는 논문만 결과로 나올 수 있게 하여 다시 시도 필요
  